In [1]:
import numpy as np
import cv2
import base64
import requests
import os
import pickle

---------------------------------
Model Convert Step

In [17]:
url_hoggen = 'http://localhost:8000/api/genhog'
cars_path = f"Plants_dataset/"
written_path = f'plant_models/'

In [18]:
def saveModel(obj, name):
    model = f'{written_path}{name}.pk'
    pickle.dump(obj, open(model, 'wb'))
    print(f'saved {model} successfully.')

def loadModel(name):
    return pickle.load(open(f'{written_path}{name}', 'rb'))

In [19]:
def getHOGData(img):
    # convert image to buffer
    v, buffer = cv2.imencode('.jpg', img)
    # convert buffer of image to base64
    img_str = "img,"+str.split(str(base64.b64encode(buffer)), "'")[1]

    req = requests.get(url_hoggen, json={"img":img_str})
    return req.json()['data']

type_dict = {
}

In [26]:
#convert cars dataset into HOG Data & save file
def convertAllAndSave(train_test=True, returned=False):
  hog_list = []
  type_list = []

  dest_path = 'train' if train_test else 'test'
  
  print(f"Converting {dest_path} plants into HOG Model...")
  i = 0
  for subFolder in os.listdir(cars_path+dest_path):
      print(f'{i}. {subFolder}')
      type_dict[subFolder] = i
      try:
          subFolderRelPath = os.path.join(cars_path+dest_path,subFolder)
          for file in os.listdir(subFolderRelPath):
            img = cv2.imread(subFolderRelPath+"/"+file)
            hog_list.append(getHOGData(img))
            type_list.append(type_dict[subFolder])
          
          i = i + 1
      except:
          print("error in dir: " + subFolder)

  print(f'\n{len(hog_list)} Model loaded complete!')
  saveModel(hog_list, ('' if train_test else 'test_') + 'plantHog')
  saveModel(type_list, ('' if train_test else 'test_') + 'plantType')

  if returned:
    return {"hog": hog_list, "type": type_list}
  else:
    return None

In [14]:

hog_list = []
type_list = []
dest_path = 'train'

type_dict = {
}

i = 0
for subFolder in os.listdir(cars_path+dest_path):
    print(f'{i}. {subFolder}')
    type_dict[subFolder] = i
    try:
        subFolderRelPath = os.path.join(cars_path+dest_path,subFolder)
        # for file in os.listdir(subFolderRelPath):
        #   img = cv2.imread(subFolderRelPath+"/"+file)
        #   hog_list.append(getHOGData(img))
          # type_list.append(type_dict[subFolder])
        
        i = i + 1
    except:
        print("error in dir: " + subFolder)

0. kale
1. sweetpotatoes
2. spinach
3. peperchili
4. classname.txt
5. tobacco
6. guava
7. bilimbi
8. aloevera
9. cucumber
10. pomelo
11. watermelon
12. cantaloupe
13. waterapple
14. curcuma
15. mango
16. coconut
17. galangal
18. eggplant
19. cassava
20. corn
21. soybeans
22. banana
23. longbeans
24. ginger
25. paddy
26. papaya
27. shallot
28. pineapple
29. melon
30. orange


Convert Model 

In [21]:
convertAllAndSave(True)

Converting train plants into HOG Model...
error in dir: classname.txt

21000 Model loaded complete!
saved plant_models/plantHog.pk successfully.
saved plant_models/plantType.pk successfully.


---------------------------------
Generate Model Step

In [23]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix

Python(34139) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


In [24]:
hog_data = loadModel('plantHog.pk')
type_data = loadModel('plantType.pk')

X_train = np.array(hog_data)
Y_train = np.array(type_data)


In [25]:
clf = DecisionTreeClassifier()
clf.fit(X_train, Y_train)

DecisionTreeClassifier()

----------------------
test session

In [27]:
hog_data = convertAllAndSave(False, True)
test_hog_list = hog_data['hog']
test_type_list = hog_data['type']

Converting test plants into HOG Model...
0. kale
1. sweetpotatoes
2. .DS_Store
error in dir: .DS_Store
2. spinach
3. peperchili
4. classname.txt
error in dir: classname.txt
4. tobacco
5. guava
6. bilimbi
7. aloevera
8. cucumber
9. pomelo
10. watermelon
11. cantaloupe
12. waterapple
13. curcuma
14. mango
15. coconut
16. galangal
17. eggplant
18. cassava
19. corn
20. soybeans
21. banana
22. longbeans
23. ginger
24. paddy
25. papaya
26. shallot
27. pineapple
28. melon
29. orange

6000 Model loaded complete!
saved plant_models/test_plantHog.pk successfully.
saved plant_models/test_plantType.pk successfully.


In [28]:
hogs_np = np.array(test_hog_list)
type_np = np.array([[label] for label in test_type_list]) #make value inside into 2D array ([1,3,2,3] => [[1],[3],[2],[3]])
X_test = hogs_np
Y_test = type_np

In [29]:
X_train, X_test, y_train, y_test = train_test_split(hogs_np, type_np, test_size=0.2, random_state=42)
# Make predictions on the test set
y_pred = clf.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)*100
print("Accuracy:", accuracy)
print("Confusion Matrix:", accuracy)
confusion_matrix(y_test, y_pred)

Accuracy: 26.166666666666664
Confusion Matrix: 26.166666666666664


array([[12,  1,  3,  1,  0,  1,  0,  0,  1,  1,  0,  1,  1,  1,  1,  1,
         2,  4,  1,  3,  0,  0,  1,  0,  4,  1,  1,  4,  0,  3],
       [ 2, 22,  0,  0,  2,  2,  1,  1,  0,  1,  1,  1,  2,  1,  0,  0,
         0,  1,  2,  0,  0,  1,  0,  1,  1,  0,  1,  0,  0,  0],
       [ 3,  0, 18,  2,  0,  2,  0,  0,  1,  5,  1,  3,  0,  1,  1,  1,
         0,  1,  1,  2,  1,  2,  0,  1,  0,  1,  1,  2,  0,  1],
       [ 1,  1,  2, 11,  0,  4,  2,  0,  2,  0,  0,  0,  1,  1,  2,  1,
         1,  2,  0,  0,  1,  2,  2,  1,  0,  2,  0,  1,  0,  1],
       [ 2,  1,  0,  0,  1,  0,  0,  0,  0,  0,  0,  3,  2,  0,  0,  0,
         2,  0,  0,  3,  2,  0,  0,  0,  4,  3,  3,  2,  0,  3],
       [ 0,  2,  0,  3,  3, 16,  2,  1,  2,  4,  1,  1,  1,  2,  2,  1,
         0,  1,  3,  1,  0,  0,  0,  1,  2,  1,  1,  1,  0,  1],
       [ 0,  0,  0,  1,  4,  1, 11,  0,  0,  1,  1,  1,  1,  1,  2,  1,
         3,  2,  1,  0,  1,  1,  2,  1,  1,  0,  1,  0,  0,  2],
       [ 2,  1,  2,  4,  1,  2,  1, 12,  

In [30]:
saveModel(clf, 'plantsModel')

saved plant_models/plantsModel.pk successfully.
